# ETS Anomaly Detection
This notebook uses ETS (Error, Trend, Seasonality) models to detect anomalies in hourly API volume data by identifying points that fall outside the model's confidence intervals.

In [ ]:
!pip install pandas numpy matplotlib statsmodels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

## Step 1: Load the Dataset

In [ ]:
# Load the dataset (replace the path with your file location)
file_path = '/path/to/api_volume_data_with_anomalies_rounded.csv'
data = pd.read_csv(file_path)

# Convert 'time' to datetime and set as index
data['time'] = pd.to_datetime(data['time'])
data.set_index('time', inplace=True)

# Preview the data
data.head()

## Step 2: Initialize and Fit the ETS Model

In [ ]:
# Initialize and fit the ETS model
model = ExponentialSmoothing(data['login_count'],
                             seasonal='add',
                             seasonal_periods=24).fit()

# Generate in-sample predictions
data['forecast'] = model.fittedvalues
data['lower_bound'] = data['forecast'] - 1.96 * model.resid.std()
data['upper_bound'] = data['forecast'] + 1.96 * model.resid.std()

## Step 3: Detect Anomalies

In [ ]:
# Detect anomalies: when actual data is outside the prediction confidence interval
data['anomaly'] = (data['login_count'] < data['lower_bound']) | (data['login_count'] > data['upper_bound'])

## Step 4: Plot the Results with Anomalies Highlighted

In [ ]:
# Plot actual data with ETS forecast and confidence intervals
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['login_count'], label='Actual Data', color='blue')
plt.plot(data.index, data['forecast'], label='Forecast', color='green')
plt.fill_between(data.index, data['lower_bound'], data['upper_bound'], color='lightgreen', alpha=0.5)

# Highlight anomalies
anomalies = data[data['anomaly']]
plt.scatter(anomalies.index, anomalies['login_count'], color='red', label='Anomalies', marker='o')

plt.xlabel('Date')
plt.ylabel('Login Count')
plt.title('API Volume Anomaly Detection with ETS')
plt.legend()
plt.show()

## Step 5: Save the Results

In [ ]:
# Save the data with anomaly indicators
output_path = '/path/to/ets_anomaly_detection_output.csv'
data[['login_count', 'forecast', 'lower_bound', 'upper_bound', 'anomaly']].to_csv(output_path, index=True)
print(f'Results saved to {output_path}')